[coda](https://www.codabench.org/competitions/2378/) [git](https://github.com/IRT-SystemX/LIPS/tree/fb4048e274fb75dd963bd538429b6ece2df757d8) [example](https://github.com/IRT-SystemX/LIPS/blob/fb4048e274fb75dd963bd538429b6ece2df757d8/getting_started/PowerGridUsecase/04_Complete_example.ipynb) [paper](https://openreview.net/pdf?id=ObD_o92z4p)

# Setup

In [1]:
!git clone https://github.com/IRT-SystemX/LIPS.git

Cloning into 'LIPS'...
remote: Enumerating objects: 7175, done.
remote: Counting objects: 100% (827/827), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 7175 (delta 713), reused 593 (delta 581), pack-reused 6348
Receiving objects: 100% (7175/7175), 628.51 MiB | 36.28 MiB/s, done.
Resolving deltas: 100% (2794/2794), done.


In [2]:
!pip install -q Grid2Op==1.9.8 leap-net==0.0.5 "numpy<1.24" lightsim2grid #==0.7.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 820.3/820.3 kB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 100.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


In [3]:
exit()

## custom model

In [4]:
%%writefile /content/LIPS/lips/augmented_simulators/tensorflow_models/transformer_utils.py
import numpy as np
import tensorflow as tf


def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x):
    heads = tf.split(x, self.num_heads, axis=-1)
    x = tf.stack(heads, axis=1)
    return x

  def call(self, v, k, q, mask=None, training=False):
    seq_len_q = tf.shape(q)[1]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v)  # (batch_size, num_heads, seq_len_v, depth)

    # q = tf.keras.layers.GaussianNoise(.1)(q, training=training)
    attention_weights = {}
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (-1, seq_len_q, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation=tf.keras.activations.gelu),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])


class ReZero(tf.keras.layers.Layer):
    def __init__(self, name):
        super(ReZero, self).__init__(name=name)
        a_init = tf.zeros_initializer()
        self.alpha = tf.Variable(name=self.name + '-alpha',
            initial_value=a_init(shape=(1,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return self.alpha * inputs


class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    #self.rz1 = ReZero(self.name+'rz1')
    #self.rz2 = ReZero(self.name+'rz2')

    #self.dropout1 = tf.keras.layers.Dropout(rate)
    #self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training=False, mask=None):

    #inp1 = x
    #x = self.layernorm1(x)
    attn_output, attention_weights = self.mha(x, x, x, mask, training=training)  # (batch_size, input_seq_len, d_model)
    #attn_output = self.dropout1(attn_output, training=training)
    #out1 = x + self.layernorm1(attn_output)  # (batch_size, input_seq_len, d_model)
    # out1 = x + self.rz1(attn_output)
    out1 = self.layernorm1(x + attn_output)

    #out1 = self.layernorm1(out1)
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    #ffn_output = self.dropout2(ffn_output, training=training)
    # out2 = out1 + self.layernorm2(ffn_output)  # (batch_size, input_seq_len, d_model)
    # out2 = out1 + self.rz2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2, attention_weights


class AttentionPool(tf.keras.layers.Layer):
    def __init__(self, d_model, dff):
        super(AttentionPool, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads=2)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        self.rz = ReZero(self.name+'rz')
        # self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, x, t, training=False):
        x, _ = self.mha(x, x, t, training=training)
        out1 = tf.math.reduce_mean(x, axis=1)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        #ffn_output = self.dropout2(ffn_output, training=training)
        # out2 = out1 + self.layernorm(ffn_output)  # (batch_size, input_seq_len, d_model)
        out2 = out1 + self.rz(ffn_output)

        return out2


class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, dff):
        super(CrossAttention, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads=2)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        self.rz = ReZero(self.name+'rz')
        # self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, q, kv, training=False):
        x, _ = self.mha(kv, kv, q, training=training)

        ffn_output = self.ffn(x)  # (batch_size, q_seq_len, d_model)
        out2 = q + self.rz(ffn_output)

        return out2


class TransformerEncoder(tf.keras.layers.Layer):
  def __init__(self, output_dim, x_attr_sizes, t_attr_sizes, num_layers, d_model, num_heads, dff):
    super(TransformerEncoder, self).__init__()

    self.attr_sizes = list(x_attr_sizes) + list(t_attr_sizes)
    self.n_tau_attr = len(t_attr_sizes)

    self.embeddings = [tf.keras.layers.Dense(d_model) for _ in self.attr_sizes]
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff)
                       for _ in range(num_layers)]

    self.pool = AttentionPool(d_model, dff)
    self.output_layer = point_wise_feed_forward_network(output_dim, dff)

  def call(self, x):
    n = self.n_tau_attr

    x = tf.split(x, self.attr_sizes, axis=1)
    x = [tf.expand_dims(lay(inp), axis=1) for (lay, inp) in zip(self.embeddings, x)]
    x, t = x[:-n], x[-n:]
    x, t = tf.concat(x, axis=1), tf.concat(t, axis=1)
    x = tf.concat([t, x], axis=1)

    x = self.ffn(x)

    for lay in self.enc_layers:
      x, _ = lay(x)

    x = self.pool(x[:, :n], x[:, :n])
    x = self.output_layer(x)
    return x


from leap_net.LtauNoAdd import LtauNoAdd


class TransformerEncoder(tf.keras.layers.Layer):
  def __init__(self, output_dim, x_attr_sizes, t_attr_sizes, y_attr_sizes,
               num_layers, d_model, num_heads, dff):
    super(TransformerEncoder, self).__init__()

    self.attr_sizes = list(x_attr_sizes) + list(t_attr_sizes)

    self.embeddings = [tf.keras.layers.Dense(d_model) for _ in self.attr_sizes]
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff)
                       for _ in range(num_layers)]
    self.leap_layer = LtauNoAdd()

    self.output_emb_layer = point_wise_feed_forward_network(d_model, dff)
    self.output_layer = point_wise_feed_forward_network(output_dim, dff)

  def call(self, x):
    x = tf.split(x, self.attr_sizes, axis=1)
    x = [tf.expand_dims(lay(inp), axis=1)
         for (lay, inp) in zip(self.embeddings, x)]
    x = tf.concat(x, axis=1)

    x = self.ffn(x)

    for lay in self.enc_layers:
      x, _ = lay(x)

    x = tf.reduce_max(x, axis=1)
    x = self.output_emb_layer(x)
    x = self.leap_layer([x, x])

    x = self.output_layer(x)
    return x


class TransformerEncoderTrans(tf.keras.layers.Layer):
  def __init__(self, output_dim, x_attr_sizes, t_attr_sizes, y_attr_sizes,
               num_layers, d_model, num_heads, dff):
    super(TransformerEncoderTrans, self).__init__()

    self.attr_sizes = list(x_attr_sizes) + list(t_attr_sizes)

    self.seq_len = len(y_attr_sizes)
    print('attr_sizes', x_attr_sizes, t_attr_sizes, y_attr_sizes)
    self.latent_emb = tf.keras.layers.Embedding(len(y_attr_sizes), d_model)
    self.cross = CrossAttention(d_model, dff)

    self.embeddings = [tf.keras.layers.Dense(d_model, activation='elu')
                       for _ in self.attr_sizes]

    self.ffn = point_wise_feed_forward_network(d_model, dff)
    self.d_model = d_model

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff)
                       for _ in range(num_layers)]

    self.dec_emb_layer = point_wise_feed_forward_network(output_dim, dff)
    self.dec_layers = [point_wise_feed_forward_network(s, 2*s)
                       for s in y_attr_sizes]

  def call(self, inp):
    split = tf.split(inp, self.attr_sizes, axis=1)
    x = [tf.expand_dims(lay(inp), axis=1)
         for (lay, inp) in zip(self.embeddings, split)]
    x = tf.concat(x, axis=1)
    x = self.ffn(x)
    print("inp", x.shape)

    lat = self.latent_emb(tf.range(self.seq_len)[None])
    x = self.cross(lat, x)

    print('cross', x.shape)
    for lay in self.enc_layers:
      x, _ = lay(x)

    x = self.dec_emb_layer(x)
    split = tf.unstack(x, axis=1)
    x = [lay(z) for (lay, z) in zip(self.dec_layers, split)]
    x = tf.concat(x, axis=-1)
    return x


Writing /content/LIPS/lips/augmented_simulators/tensorflow_models/transformer_utils.py


In [5]:
%%writefile /content/LIPS/lips/augmented_simulators/tensorflow_models/transformer.py

# Copyright (c) 2021, IRT SystemX (https://www.irt-systemx.fr/en/)
# See AUTHORS.txt
# This Source Code Form is subject to the terms of the Mozilla Public License, version 2.0.
# If a copy of the Mozilla Public License, version 2.0 was not distributed with this file,
# you can obtain one at http://mozilla.org/MPL/2.0/.
# SPDX-License-Identifier: MPL-2.0
# This file is part of LIPS, LIPS is a python platform for power networks benchmarking

import os
import pathlib
from typing import Union
import json
import warnings

import numpy as np
# from leap_net import ResNetLayer

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    from tensorflow import keras

from ..tensorflow_simulator import TensorflowSimulator
from ...logger import CustomLogger
from ...config import ConfigManager
from ...dataset import DataSet
from ...dataset.scaler import Scaler
from ...utils import NpEncoder

from .transformer_utils import TransformerEncoder, TransformerEncoderTrans


class SimpNet(TensorflowSimulator):
    """Fully Connected architecture
    Parameters
    ----------
    sim_config_path : ``str``
        The path to the configuration file for simulator.
        It should contain all the required hyperparameters for this model.
    sim_config_name : Union[str, None], optional
        _description_, by default None
    name : Union[str, None], optional
        _description_, by default None
    scaler : Union[Scaler, None], optional
        _description_, by default None
    bench_config_path : Union[str, pathlib.Path, None], optional
        _description_, by default None
    bench_config_name : Union[str, None], optional
        _description_, by default None
    log_path : Union[None, str], optional
        _description_, by default None
    Raises
    ------
    RuntimeError
        _description_
    """
    def __init__(self,
                 sim_config_path: str,
                 bench_config_path: Union[str, pathlib.Path],
                 bench_config_name: Union[str, None]=None,
                 bench_kwargs: dict={},
                 sim_config_name: Union[str, None]=None,
                 name: Union[str, None]=None,
                 scaler: Union[Scaler, None]=None,
                 log_path: Union[None, str]=None,
                 **kwargs):
        super().__init__(name=name, log_path=log_path, **kwargs)
        if not os.path.exists(sim_config_path):
            raise RuntimeError("Configuration path for the simulator not found!")
        if not str(sim_config_path).endswith(".ini"):
            raise RuntimeError("The configuration file should have `.ini` extension!")
        sim_config_name = sim_config_name if sim_config_name is not None else "DEFAULT"
        self.sim_config = ConfigManager(section_name=sim_config_name, path=sim_config_path)
        self.bench_config = ConfigManager(section_name=bench_config_name, path=bench_config_path)
        self.bench_config.set_options_from_dict(**bench_kwargs)
        self.name = name if name is not None else self.sim_config.get_option("name")
        self.name = self.name + '_' + sim_config_name
        # scaler
        self.scaler = scaler() if scaler else None
        # Logger
        self.log_path = log_path
        self.logger = CustomLogger(__class__.__name__, log_path).logger
        # model parameters
        self.params = self.sim_config.get_options_dict()
        self.params.update(kwargs)
        # Define layer to be used for the model
        self.layers = {"linear": keras.layers.Dense}
        self.layer = self.layers.get(self.params["layer"], None)
        if self.layer is None:
            self.layer = keras.layers.Dense

        # optimizer
        if "lr" in kwargs:
            if not isinstance(kwargs["lr"], float):
                raise RuntimeError("Learning rate (lr) is provided, it should be a float")
            lr = kwargs['lr']
        else:
            lr = self.params["optimizer"]["params"]["lr"]
        print('lr', lr)
        self._optimizer = keras.optimizers.Adam(learning_rate=lr)

        self._model: Union[keras.Model, None] = None

        self.input_size = None if kwargs.get("input_size") is None else kwargs["input_size"]
        self.output_size = None if kwargs.get("output_size") is None else kwargs["output_size"]

        self.x_attr_sizes, self.t_attr_sizes, self.y_attr_sizes = None, None, None

    def build_model(self):
        """Build the model
        Returns
        -------
        Model
            _description_
        """
        super().build_model()
        transformer = TransformerEncoder(
            self.output_size, self.x_attr_sizes, self.t_attr_sizes, self.y_attr_sizes,
            num_layers=5, d_model=512, num_heads=8, dff=512)
        input_ = keras.layers.Input(shape=(self.input_size,), name="input")
        output_ = transformer(input_)
        self._model = keras.Model(inputs=input_,
                                  outputs=output_,
                                  name=f"{self.name}_model")
        return self._model

    def process_dataset(self, dataset: DataSet, training: bool=False) -> tuple:
        """process the datasets for training and evaluation
        This function transforms all the dataset into something that can be used by the neural network (for example)
        Warning
        -------
        It works with StandardScaler only for the moment.
        Parameters
        ----------
        dataset : DataSet
            _description_
        Scaler : bool, optional
            _description_, by default True
        training : bool, optional
            _description_, by default False
        Returns
        -------
        tuple
            the normalized dataset with features and labels
        """
        if training:
            self._infer_size(dataset)
            inputs, outputs = dataset.extract_data(concat=False)

            inputs, outputs = dataset.extract_data(concat=True)
            if self.scaler is not None:
                inputs, outputs = self.scaler.fit_transform(inputs, outputs)
        else:
            inputs, outputs = dataset.extract_data(concat=True)
            if self.scaler is not None:
                inputs, outputs = self.scaler.transform(inputs, outputs)

        self.x_attr_sizes, self.t_attr_sizes, self.y_attr_sizes = dataset._sizes_x, dataset._sizes_tau, dataset._sizes_y
        return inputs, outputs

    def _infer_size(self, dataset: DataSet):
        """Infer the size of the model
        Parameters
        ----------
        dataset : DataSet
            _description_
        Returns
        -------
        None
            _description_
        """
        *dim_inputs, self.output_size = dataset.get_sizes()
        self.input_size = np.sum(dim_inputs)

    def _post_process(self, dataset, predictions):
        if self.scaler is not None:
            predictions = self.scaler.inverse_transform(predictions)
        predictions = super()._post_process(dataset, predictions)
        return predictions

    def _save_metadata(self, path: str):
        super()._save_metadata(path)
        if self.scaler is not None:
            self.scaler.save(path)
        res_json = {}
        res_json["input_size"] = self.input_size
        res_json["output_size"] = self.output_size
        with open((path / "metadata.json"), "w", encoding="utf-8") as f:
            json.dump(obj=res_json, fp=f, indent=4, sort_keys=True, cls=NpEncoder)

    def _load_metadata(self, path: str):
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        super()._load_metadata(path)
        if self.scaler is not None:
            self.scaler.load(path)
        with open((path / "metadata.json"), "r", encoding="utf-8") as f:
            res_json = json.load(fp=f)
        self.input_size = res_json["input_size"]
        self.output_size = res_json["output_size"]


Writing /content/LIPS/lips/augmented_simulators/tensorflow_models/transformer.py


In [6]:
%%writefile /content/LIPS/lips/augmented_simulators/tensorflow_models/simpnet.py

# Copyright (c) 2021, IRT SystemX (https://www.irt-systemx.fr/en/)
# See AUTHORS.txt
# This Source Code Form is subject to the terms of the Mozilla Public License, version 2.0.
# If a copy of the Mozilla Public License, version 2.0 was not distributed with this file,
# you can obtain one at http://mozilla.org/MPL/2.0/.
# SPDX-License-Identifier: MPL-2.0
# This file is part of LIPS, LIPS is a python platform for power networks benchmarking

import os
import pathlib
from typing import Union
import json
import warnings

import numpy as np
# from leap_net import ResNetLayer

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    from tensorflow import keras

from ..tensorflow_simulator import TensorflowSimulator
from ...logger import CustomLogger
from ...config import ConfigManager
from ...dataset import DataSet
from ...dataset.scaler import Scaler
from ...utils import NpEncoder


class SimpNet(TensorflowSimulator):
    """Fully Connected architecture
    Parameters
    ----------
    sim_config_path : ``str``
        The path to the configuration file for simulator.
        It should contain all the required hyperparameters for this model.
    sim_config_name : Union[str, None], optional
        _description_, by default None
    name : Union[str, None], optional
        _description_, by default None
    scaler : Union[Scaler, None], optional
        _description_, by default None
    bench_config_path : Union[str, pathlib.Path, None], optional
        _description_, by default None
    bench_config_name : Union[str, None], optional
        _description_, by default None
    log_path : Union[None, str], optional
        _description_, by default None
    Raises
    ------
    RuntimeError
        _description_
    """
    def __init__(self,
                 sim_config_path: str,
                 bench_config_path: Union[str, pathlib.Path],
                 bench_config_name: Union[str, None]=None,
                 bench_kwargs: dict={},
                 sim_config_name: Union[str, None]=None,
                 name: Union[str, None]=None,
                 scaler: Union[Scaler, None]=None,
                 log_path: Union[None, str]=None,
                 **kwargs):
        super().__init__(name=name, log_path=log_path, **kwargs)
        if not os.path.exists(sim_config_path):
            raise RuntimeError("Configuration path for the simulator not found!")
        if not str(sim_config_path).endswith(".ini"):
            raise RuntimeError("The configuration file should have `.ini` extension!")
        sim_config_name = sim_config_name if sim_config_name is not None else "DEFAULT"
        self.sim_config = ConfigManager(section_name=sim_config_name, path=sim_config_path)
        self.bench_config = ConfigManager(section_name=bench_config_name, path=bench_config_path)
        self.bench_config.set_options_from_dict(**bench_kwargs)
        self.name = name if name is not None else self.sim_config.get_option("name")
        self.name = self.name + '_' + sim_config_name
        # scaler
        self.scaler = scaler() if scaler else None
        # Logger
        self.log_path = log_path
        self.logger = CustomLogger(__class__.__name__, log_path).logger
        # model parameters
        self.params = self.sim_config.get_options_dict()
        self.params.update(kwargs)
        # Define layer to be used for the model
        self.layers = {"linear": keras.layers.Dense}
        self.layer = self.layers.get(self.params["layer"], None)
        if self.layer is None:
            self.layer = keras.layers.Dense

        # optimizer
        if "lr" in kwargs:
            if not isinstance(kwargs["lr"], float):
                raise RuntimeError("Learning rate (lr) is provided, it should be a float")
            lr = kwargs['lr']
        else:
            lr = self.params["optimizer"]["params"]["lr"]
        print('lr', lr)
        self._optimizer = keras.optimizers.Adam(learning_rate=lr)

        self._model: Union[keras.Model, None] = None

        self.input_size = None if kwargs.get("input_size") is None else kwargs["input_size"]
        self.output_size = None if kwargs.get("output_size") is None else kwargs["output_size"]

    def build_model(self):
        """Build the model
        Returns
        -------
        Model
            _description_
        """
        super().build_model()
        input_ = keras.layers.Input(shape=(self.input_size,), name="input")
        x = input_
        # x = keras.layers.Dropout(rate=self.params["input_dropout"], name="input_dropout")(x)
        for layer_id, layer_size in enumerate(self.params["layers"]):
            x = self.layer(layer_size, name=f"layer_{layer_id}")(x)
            x = keras.layers.LayerNormalization(name=f"norm_{layer_id}")(x)
            x = keras.layers.Activation(self.params["activation"], name=f"activation_{layer_id}")(x)
            # x = keras.layers.Activation(self.params["activation"], name=f"activation_{layer_id}")(x)
            # x = keras.layers.Dropout(rate=self.params["dropout"], name=f"dropout_{layer_id}")(x)

        output_ = keras.layers.Dense(self.output_size)(x)
        self._model = keras.Model(inputs=input_,
                                  outputs=output_,
                                  name=f"{self.name}_model")
        return self._model

    def process_dataset(self, dataset: DataSet, training: bool=False) -> tuple:
        """process the datasets for training and evaluation
        This function transforms all the dataset into something that can be used by the neural network (for example)
        Warning
        -------
        It works with StandardScaler only for the moment.
        Parameters
        ----------
        dataset : DataSet
            _description_
        Scaler : bool, optional
            _description_, by default True
        training : bool, optional
            _description_, by default False
        Returns
        -------
        tuple
            the normalized dataset with features and labels
        """
        if training:
            self._infer_size(dataset)
            inputs, outputs = dataset.extract_data(concat=False)

            inputs, outputs = dataset.extract_data(concat=True)
            if self.scaler is not None:
                inputs, outputs = self.scaler.fit_transform(inputs, outputs)
        else:
            inputs, outputs = dataset.extract_data(concat=True)
            if self.scaler is not None:
                inputs, outputs = self.scaler.transform(inputs, outputs)

        return inputs, outputs

    def _infer_size(self, dataset: DataSet):
        """Infer the size of the model
        Parameters
        ----------
        dataset : DataSet
            _description_
        Returns
        -------
        None
            _description_
        """
        *dim_inputs, self.output_size = dataset.get_sizes()
        self.input_size = np.sum(dim_inputs)

    def _post_process(self, dataset, predictions):
        if self.scaler is not None:
            predictions = self.scaler.inverse_transform(predictions)
        predictions = super()._post_process(dataset, predictions)
        return predictions

    def _save_metadata(self, path: str):
        super()._save_metadata(path)
        if self.scaler is not None:
            self.scaler.save(path)
        res_json = {}
        res_json["input_size"] = self.input_size
        res_json["output_size"] = self.output_size
        with open((path / "metadata.json"), "w", encoding="utf-8") as f:
            json.dump(obj=res_json, fp=f, indent=4, sort_keys=True, cls=NpEncoder)

    def _load_metadata(self, path: str):
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        super()._load_metadata(path)
        if self.scaler is not None:
            self.scaler.load(path)
        with open((path / "metadata.json"), "r", encoding="utf-8") as f:
            res_json = json.load(fp=f)
        self.input_size = res_json["input_size"]
        self.output_size = res_json["output_size"]


Writing /content/LIPS/lips/augmented_simulators/tensorflow_models/simpnet.py


In [7]:
%%writefile /content/LIPS/lips/augmented_simulators/tensorflow_simulator.py
# Copyright (c) 2021, IRT SystemX (https://www.irt-systemx.fr/en/)
# See AUTHORS.txt
# This Source Code Form is subject to the terms of the Mozilla Public License, version 2.0.
# If a copy of the Mozilla Public License, version 2.0 was not distributed with this file,
# you can obtain one at http://mozilla.org/MPL/2.0/.
# SPDX-License-Identifier: MPL-2.0
# This file is part of LIPS, LIPS is a python platform for power networks benchmarking

import os
import pathlib
from typing import Union
import shutil
import json
import tempfile
import importlib

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

from . import AugmentedSimulator
from ..utils import NpEncoder
from ..dataset import DataSet
from ..logger import CustomLogger


class TensorflowSimulator(AugmentedSimulator):
    """_summary_

        Parameters
        ----------
        name : str, optional
            _description_, by default None
        config : ConfigManager
            _description_
        """
    def __init__(self,
                 name: Union[str, None]=None,
                 log_path: Union[str, None] = None,
                 **kwargs):
        super().__init__(name=name, log_path=log_path, **kwargs)
        # logger
        self.logger = CustomLogger(__class__.__name__, self.log_path).logger
        self._optimizer = None

        self.input_size = None
        self.output_size = None

        # setting seeds
        np.random.seed(1)
        # tf.random.set_seed(2)


    def build_model(self):
        """build tensorflow model

        Parameters
        ----------
        **kwargs : dict
            if parameters indicated, it will replace config parameters

        Returns
        -------
        keras.Model
            _description_
        """
        if self.input_size is None or self.output_size is None:
            raise RuntimeError("input_size is not set")


    def train(self,
              train_dataset: DataSet,
              val_dataset: Union[None, DataSet] = None,
              save_path: Union[None, str] = None,
              **kwargs):
        """Function used to train a neural network

        Parameters
        ----------
        train_dataset : DataSet
            training dataset
        val_dataset : Union[None, DataSet], optional
            validation dataset, by default None
        save_path : Union[None, str], optional
            the path where the trained model should be saved, by default None
            #TODO: a callback for tensorboard and another for saving the model
        """
        super().train(train_dataset, val_dataset)
        self.params.update(kwargs)
        processed_x, processed_y = self.process_dataset(train_dataset, training=True)

        if val_dataset is not None:
            processed_x_val, processed_y_val = self.process_dataset(val_dataset, training=False)
            validation_data = (processed_x_val, processed_y_val)
        else:
            validation_data = None

        # init the model
        self.build_model()

        self._model.compile(optimizer=self._optimizer,
                            loss="mae", # self.params["loss"]["name"],
                            metrics=self.params["metrics"])

        cb = [
            tf.keras.callbacks.ModelCheckpoint(
                "temp.keras",
                monitor="val_mae",
                verbose=1,
                save_best_only=True,
                save_weights_only=True)
            ]

        self.logger.info("Training of {%s} started", self.name)
        history_callback = self._model.fit(x=processed_x,
                                           y=processed_y,
                                           validation_data=validation_data,
                                           epochs=self.params["epochs"],
                                           batch_size=self.params["train_batch_size"],
                                           shuffle=self.params["shuffle"],
                                           callbacks=cb,
                                           verbose=2)
        self.logger.info("Training of {%s} finished", self.name)
        self.write_history(history=history_callback, val_dataset=validation_data)
        self.trained = True
        self._model.load_weights("temp.keras")
        os.remove("temp.keras")

        if save_path is not None:
            self.save(save_path)

        return history_callback

    def predict(self, dataset: DataSet, **kwargs) -> dict:
        """_summary_

        Parameters
        ----------
        dataset : DataSet
            test datasets to evaluate
        """
        super().predict(dataset)

        if "eval_batch_size" in kwargs:
            self.params["eval_batch_size"] = kwargs["eval_batch_size"]
        # self.params.update(kwargs)

        #processed_x, processed_y = self._process_all_dataset(dataset, training=False)
        processed_x, _ = self.process_dataset(dataset, training=False)

        # make the predictions
        predictions = self._model.predict(processed_x, batch_size=self.params["eval_batch_size"])

        predictions = self._post_process(dataset, predictions)

        self._predictions[dataset.name] = predictions
        self._observations[dataset.name] = dataset.data

        return predictions

    def process_dataset(self, dataset: DataSet, training: bool) -> tuple:
        """process the datasets for training and evaluation

        each augmented simulator requires its owan data preparation

        This function transforms all the dataset into something that can be used by the neural network (for example)

        Parameters
        ----------
        dataset : DataSet
            _description_
        training : bool, optional
            _description_, by default False

        Returns
        -------
        tuple
            the normalized dataset with features and labels
        """
        super().process_dataset(dataset, training)
        inputs, outputs = dataset.extract_data()

        return inputs, outputs

    def _post_process(self, dataset, predictions):
        """Do some post processing on the predictions

        Parameters
        ----------
        predictions : _type_
            _description_

        Returns
        -------
        _type_
            _description_
        """
        return dataset.reconstruct_output(predictions)


    ###############################################
    # function used to save and restore the model #
    ###############################################
    def save(self, path: str, save_metadata: bool=True):
        """_summary_

        Parameters
        ----------
        path : str
            _description_
        save_metadata : bool, optional
            _description_, by default True
        """
        save_path =  pathlib.Path(path) / self.name
        super().save(save_path)

        self._save_model(save_path)

        if save_metadata:
            self._save_metadata(save_path)

        self.logger.info("Model {%s} is saved at {%s}", self.name, save_path)

    def _save_model(self, path: Union[str, pathlib.Path], ext: str=".h5"):
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        file_name = path / ("weights" + ext)
        self._model.save_weights(file_name)

    def _save_metadata(self, path: Union[str, pathlib.Path]):
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        # for json serialization of paths
        #pydantic.json.ENCODERS_BY_TYPE[pathlib.PosixPath] = str
        #pydantic.json.ENCODERS_BY_TYPE[pathlib.WindowsPath] = str
        self._save_losses(path)
        with open((path / "config.json"), "w", encoding="utf-8") as f:
            json.dump(obj=self.params, fp=f, indent=4, sort_keys=True, cls=NpEncoder)

    def restore(self, path: str):
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        full_path = path / self.name
        if not full_path.exists():
            raise FileNotFoundError(f"path {full_path} not found")
        # load the metadata
        self._load_metadata(full_path)
        self._load_model(full_path)

        self.logger.info("Model {%s} is loaded from {%s}", self.name, full_path)

    def _load_model(self, path: str):
        nm_file = "weights.h5"
        path_weights = path / nm_file
        if not path_weights.exists():
            raise FileNotFoundError(f"Weights file {path_weights} not found")
        self.build_model()
        # load the weights
        with tempfile.TemporaryDirectory() as path_tmp:
            nm_tmp = os.path.join(path_tmp, nm_file)
            # copy the weights into this file
            shutil.copy(path_weights, nm_tmp)
            # load this copy (make sure the proper file is not corrupted even if the loading fails)
            self._model.load_weights(nm_tmp)

    def _load_metadata(self, path: str):
        """
        load the model metadata
        """
        # load scaler parameters
        #self.scaler.load(full_path)
        self._load_losses(path)
        with open((path / "config.json"), "r", encoding="utf-8") as f:
            res_json = json.load(fp=f)
        self.params.update(res_json)
        return self.params

    def _save_losses(self, path: Union[str, pathlib.Path]):
        """
        save the losses
        """
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        res_losses = {}
        res_losses["train_losses"] = self.train_losses
        res_losses["train_metrics"] = self.train_metrics
        res_losses["val_losses"] = self.val_losses
        res_losses["val_metrics"] = self.val_metrics
        with open((path / "losses.json"), "w", encoding="utf-8") as f:
            json.dump(obj=res_losses, fp=f, indent=4, sort_keys=True, cls=NpEncoder)

    def _load_losses(self, path: Union[str, pathlib.Path]):
        """
        load the losses
        """
        if not isinstance(path, pathlib.Path):
            path = pathlib.Path(path)
        with open((path / "losses.json"), "r", encoding="utf-8") as f:
            res_losses = json.load(fp=f)
        self.train_losses = res_losses["train_losses"]
        self.train_metrics = res_losses["train_metrics"]
        self.val_losses = res_losses["val_losses"]
        self.val_metrics = res_losses["val_metrics"]

    #########################
    # Some Helper functions #
    #########################
    def summary(self):
        """summary of the model
        """
        print(self._model.summary())

    def plot_model(self, path: Union[str, None]=None, file_name: str="model"):
        """Plot the model architecture using GraphViz Library

        """
        # verify if GraphViz and pydot are installed
        pydot_found = importlib.util.find_spec("pydot")
        graphviz_found = importlib.util.find_spec("graphviz")
        if pydot_found is None or graphviz_found is None:
            raise RuntimeError("pydot and graphviz are required to use this function")

        if not pathlib.Path(path).exists():
            pathlib.Path(path).mkdir(parents=True, exist_ok=True)

        tf.keras.utils.plot_model(
            self._model,
            to_file=file_name+".png",
            show_shapes=True,
            show_dtype=True,
            show_layer_names=True,
            rankdir="TB",
            expand_nested=False,
            dpi=56,
            layer_range=None,
            show_layer_activations=False,
        )

    def write_history(self, history: dict, val_dataset=None):
        """write the history of the training

        Parameters
        ----------
        history_callback : keras.callbacks.History
            the history of the training
        """
        self.train_losses = history.history["loss"]
        if val_dataset is not None:
            self.val_losses = history.history["val_loss"]

        for metric in self.params["metrics"]:
            self.train_metrics[metric] = history.history[metric]
            if val_dataset is not None:
                self.val_metrics[metric] = history.history["val_" + metric]

    def count_parameters(self):
        """count the number of parameters of the model

        Returns
        -------
        int
            the number of parameters
        """
        return self._model.count_params()

    def visualize_convergence(self, figsize=(15,5), save_path: str=None):
        """Visualizing the convergence of the model
        """
        # raise an error if the train_losses is empty
        if len(self.train_losses) == 0:
            raise RuntimeError("The model should be trained before visualizing the convergence")
        num_metrics = len(self.params["metrics"])
        if num_metrics == 0:
            nb_subplots = 1
        else:
            nb_subplots = num_metrics + 1
        fig, ax = plt.subplots(1,nb_subplots, figsize=figsize)
        ax[0].set_title("MSE")
        ax[0].plot(self.train_losses, label='train_loss')
        if len(self.val_losses) > 0:
            ax[0].plot(self.val_losses, label='val_loss')
        for idx_, metric_name in enumerate(self.params["metrics"]):
            ax[idx_+1].set_title(metric_name)
            ax[idx_+1].plot(self.train_metrics[metric_name], label=f"train_{metric_name}")
            if len(self.val_metrics[metric_name]) > 0:
                ax[idx_+1].plot(self.val_metrics[metric_name], label=f"val_{metric_name}")
        for i in range(nb_subplots):
            ax[i].grid()
            ax[i].legend()
        # save the figure
        if save_path is not None:
            if not pathlib.Path(save_path).exists():
                pathlib.Path(save_path).mkdir(parents=True, exist_ok=True)
            fig.savefig(save_path)


Overwriting /content/LIPS/lips/augmented_simulators/tensorflow_simulator.py


# run

In [ ]:
import os; os.chdir('/content/LIPS')
import pathlib
from lips import get_root_path
from pprint import pprint
from matplotlib import pyplot as plt
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

from lips.augmented_simulators.tensorflow_models import TfFullyConnected
#from lips.augmented_simulators.tensorflow_models.simpnet import SimpNet
from lips.augmented_simulators.tensorflow_models.transformer import SimpNet
from lips.dataset.scaler import StandardScaler

from lips.benchmark.powergridBenchmark import PowerGridBenchmark

LIPS_PATH = pathlib.Path('/content/LIPS')
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

SAVE_PATH = "/content/drive/MyDrive/Coda/POWERGRID/dataset/"

In [ ]:
# config
benchmark_name = "Benchmark2" # @param ["Benchmark1", "Benchmark2", "Benchmark3"]
debug = False

## generate dataset

In [ ]:
benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                                benchmark_name=benchmark_name,
                                load_data_set=False,
                                config_path=BENCH_CONFIG_PATH,
                                log_path=LOG_PATH)

In [ ]:
if debug:
  benchmark.generate(nb_sample_train=100,
                      nb_sample_val=10,
                      nb_sample_test=10,
                      nb_sample_test_ood_topo=10,
                    )
else:
  benchmark.generate(nb_sample_train=int(1e5),
                      nb_sample_val=int(1e4),
                      nb_sample_test=int(1e4),
                      nb_sample_test_ood_topo=int(1e4),
                    )

# save

In [ ]:
import pickle

with open(SAVE_PATH + f'{benchmark_name}.pkl', 'wb') as f:
  pickle.dump(benchmark, f)

AttributeError: Can't pickle local object 'Grid2opSimulator.__init__.<locals>.<lambda>'

# load

In [ ]:
import pickle

with open(SAVE_PATH + f'{benchmark_name}.pkl', 'rb') as f:
  benchmark = pickle.load(f)

# train

In [ ]:
benchmark = PowerGridBenchmark(benchmark_name=benchmark_name,
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [ ]:
from lips.augmented_simulators.tensorflow_models import TfFullyConnected


tf_fc = SimpNet(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name=benchmark_name,
                         sim_config_path=SIM_CONFIG_PATH / "tf_fc.ini",
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH,
                lr= 1e-4,
                )

lr 0.0001


In [ ]:
tf_fc.train(train_dataset=benchmark.train_dataset,
            val_dataset=benchmark.val_dataset,
            epochs=1 if debug else 100
           )

Epoch 1/100

Epoch 1: val_mae improved from inf to 0.09377, saving model to temp.keras
782/782 - 46s - loss: 0.1793 - mae: 0.1793 - val_loss: 0.0938 - val_mae: 0.0938 - 46s/epoch - 59ms/step
Epoch 2/100

Epoch 2: val_mae improved from 0.09377 to 0.05271, saving model to temp.keras
782/782 - 27s - loss: 0.0593 - mae: 0.0593 - val_loss: 0.0527 - val_mae: 0.0527 - 27s/epoch - 35ms/step
Epoch 3/100

Epoch 3: val_mae improved from 0.05271 to 0.03815, saving model to temp.keras
782/782 - 27s - loss: 0.0392 - mae: 0.0392 - val_loss: 0.0382 - val_mae: 0.0382 - 27s/epoch - 35ms/step
Epoch 4/100

Epoch 4: val_mae improved from 0.03815 to 0.03150, saving model to temp.keras
782/782 - 27s - loss: 0.0307 - mae: 0.0307 - val_loss: 0.0315 - val_mae: 0.0315 - 27s/epoch - 34ms/step
Epoch 5/100

Epoch 5: val_mae improved from 0.03150 to 0.02742, saving model to temp.keras
782/782 - 27s - loss: 0.0262 - mae: 0.0262 - val_loss: 0.0274 - val_mae: 0.0274 - 27s/epoch - 35ms/step
Epoch 6/100

Epoch 6: val_mae

# benchmark scroing

In [1]:
import os; os.chdir('/content/')
!git clone https://github.com/IRT-SystemX/ml4physim_startingkit_powergrid.git

Cloning into 'ml4physim_startingkit_powergrid'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (172/172), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 527 (delta 83), reused 134 (delta 49), pack-reused 355
Receiving objects: 100% (527/527), 106.37 MiB | 31.62 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [2]:
import os; os.chdir('/content/LIPS')
!pip -q install .
exit()

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 82.6 MB/s eta 0:00:00


In [1]:
import os; os.chdir('/content/ml4physim_startingkit_powergrid')
from lips.benchmark.powergridBenchmark import PowerGridBenchmark

#Define the required paths
BENCH_CONFIG_PATH = os.path.join("configs", "benchmarks", "lips_idf_2023.ini")
DATA_PATH = os.path.join("input_data_local", "lips_idf_2023")
TRAINED_MODELS = os.path.join("input_data_local", "trained_models")
LOG_PATH = "logs.log"

benchmark_kwargs = {"attr_x": ("prod_p", "prod_v", "load_p", "load_q"),
                    "attr_y": ("a_or", "a_ex", "p_or", "p_ex", "v_or", "v_ex"),
                    "attr_tau": ("line_status", "topo_vect"),
                    "attr_physics": None}

benchmark = PowerGridBenchmark(benchmark_path=DATA_PATH,
                               config_path=BENCH_CONFIG_PATH,
                               benchmark_name="Benchmark_competition",
                               load_data_set=True,
                               log_path=LOG_PATH,
                               **benchmark_kwargs)

In [ ]:
mul = 1
benchmark.generate(nb_sample_train=int(1e5) * mul,
                    nb_sample_val=int(1e4) * mul,
                    nb_sample_test=int(1e4) * mul,
                    nb_sample_test_ood_topo=int(1e4) * mul,
                  )

downloading the training data, this may take a while.


073fb805-76e5-423d-97e3-83a62a7ac0d0:   4%|▎         | 199M/5.36G [01:22<14:38, 5.87MB/s]

In [2]:
from lips.augmented_simulators.tensorflow_models.transformer import SimpNet
from lips.dataset.scaler import StandardScaler

# Indicate the path required for corresponding augmented simulator parameters
SIM_CONFIG_PATH = os.path.join("configs", "simulators", "tf_fc.ini")

tf_fc = SimpNet(name="tf_fc",
                         bench_config_path=BENCH_CONFIG_PATH,
                         bench_config_name="Benchmark_competition",
                         bench_kwargs=benchmark_kwargs,
                         sim_config_path=SIM_CONFIG_PATH,
                         sim_config_name="DEFAULT",
                         scaler=StandardScaler,
                         log_path=LOG_PATH,
                lr= 1e-4)

#LOAD_PATH = os.path.join(TRAINED_MODELS, "lips_idf_2023")
#tf_fc.restore(path=LOAD_PATH)

lr 0.0001


In [ ]:
tf_fc.train(train_dataset=benchmark.train_dataset,
            val_dataset=benchmark.val_dataset,
            epochs=100
           )

In [ ]:
EVALUATION_PATH = os.path.join("input_data_local", "eval_results", "lips_idf_2023")
metrics = benchmark.evaluate_simulator(augmented_simulator=tf_fc,
                                       eval_batch_size=128,
                                       dataset="all",
                                       shuffle=False,
                                       save_path=EVALUATION_PATH,
                                       save_predictions=False)


In [ ]:
import math
from lips.metrics.power_grid.compute_solver_time import compute_solver_time
from lips.metrics.power_grid.compute_solver_time_grid2op import compute_solver_time_grid2op

thresholds={"a_or":(0.02,0.05,"min"),
            "a_ex":(0.02,0.05,"min"),
            "p_or":(0.02,0.05,"min"),
            "p_ex":(0.02,0.05,"min"),
            "v_or":(0.2,0.5,"min"),
            "v_ex":(0.2,0.5,"min"),
            "CURRENT_POS":(1., 5.,"min"),
            "VOLTAGE_POS":(1.,5.,"min"),
            "LOSS_POS":(1.,5.,"min"),
            "DISC_LINES":(1.,5.,"min"),
            "CHECK_LOSS":(1.,5.,"min"),
            "CHECK_GC":(0.05,0.10,"min"),
            "CHECK_LC":(0.05,0.10,"min"),
            "CHECK_JOULE_LAW":(1.,5.,"min")
           }

configuration={
    "coefficients":{"test":0.3, "test_ood":0.3, "speed_up":0.4},
    "test_ratio":{"ml": 0.66, "physics":0.34},
    "test_ood_ratio":{"ml": 0.66, "physics":0.34},
    "value_by_color":{"g":2,"o":1,"r":0},
    "max_speed_ratio_allowed":50
}

def evaluate_model(benchmark, model):
    metrics = benchmark.evaluate_simulator(augmented_simulator=model,
                                           eval_batch_size=128,
                                           dataset="all",
                                           shuffle=False,
                                           save_path=None,
                                           save_predictions=False
                                          )
    return metrics

def compute_speed_up(config, metrics):
    # solver_time = compute_solver_time(nb_samples=int(1e5), config=config)
    solver_time = compute_solver_time_grid2op(config_path=config.path_config, benchmark_name=config.section_name, nb_samples=int(1e5))
    speed_up = solver_time / metrics["test"]["ML"]["TIME_INF"]
    return speed_up

def reconstruct_metric_dict(metrics, dataset: str="test"):
    rec_metrics = dict()
    rec_metrics["ML"] = dict()
    rec_metrics["Physics"] = dict()

    rec_metrics["ML"]["a_or"] = metrics[dataset]["ML"]["MAPE_90_avg"]["a_or"]
    rec_metrics["ML"]["a_ex"] = metrics[dataset]["ML"]["MAPE_90_avg"]["a_ex"]
    rec_metrics["ML"]["p_or"] = metrics[dataset]["ML"]["MAPE_10_avg"]["p_or"]
    rec_metrics["ML"]["p_ex"] = metrics[dataset]["ML"]["MAPE_10_avg"]["p_ex"]
    rec_metrics["ML"]["v_or"] = metrics[dataset]["ML"]["MAE_avg"]["v_or"]
    rec_metrics["ML"]["v_ex"] = metrics[dataset]["ML"]["MAE_avg"]["v_ex"]

    # print(metrics[dataset]["Physics"])

    rec_metrics["Physics"]["CURRENT_POS"]     = metrics[dataset]["Physics"]["CURRENT_POS"]["a_or"]["Violation_proportion"] * 100.
    # rec_metrics["Physics"]["VOLTAGE_POS"]     = metrics[dataset]["Physics"]["VOLTAGE_POS"]["v_or"]["Violation_proportion"] * 100.
    rec_metrics["Physics"]["LOSS_POS"]        = metrics[dataset]["Physics"]["LOSS_POS"]["violation_proportion"] * 100.
    rec_metrics["Physics"]["DISC_LINES"]      = metrics[dataset]["Physics"]["DISC_LINES"]["violation_proportion"] * 100.
    rec_metrics["Physics"]["CHECK_LOSS"]      = metrics[dataset]["Physics"]["CHECK_LOSS"]["violation_percentage"]
    rec_metrics["Physics"]["CHECK_GC"]        = metrics[dataset]["Physics"]["CHECK_GC"]["violation_percentage"]
    rec_metrics["Physics"]["CHECK_LC"]        = metrics[dataset]["Physics"]["CHECK_LC"]["violation_percentage"]
    rec_metrics["Physics"]["CHECK_JOULE_LAW"] = metrics[dataset]["Physics"]["CHECK_JOULE_LAW"]["violation_proportion"] * 100.

    return rec_metrics

def discretize_results(metrics):
    results=dict()
    for subcategoryName, subcategoryVal in metrics.items():
        results[subcategoryName]=[]
        for variableName, variableError in subcategoryVal.items():
            thresholdMin,thresholdMax,evalType=thresholds[variableName]
            if evalType=="min":
                if variableError<thresholdMin:
                    accuracyEval="g"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="r"
            elif evalType=="max":
                if variableError<thresholdMin:
                    accuracyEval="r"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="g"

            results[subcategoryName].append(accuracyEval)
    return results

# def SpeedMetric(speedUp,speedMax):
#     return max(min(math.log10(speedUp)/math.log10(speedMax),1),0)

# def SpeedMetric(speedUp, speedMax):
#     a=0.01 # 0.01
#     b=0.5 #0.5
#     c=0.1 #0.1
#     k=9
#     res = quadratic_function(speedUp, a=a, b=b, c=c, k=k) / quadratic_function(speedMax, a=a, b=b, c=c, k=k)
#     return max(min(res, 1), 0)

def quadratic_function(x, a, b, c, k):
    if x == 1.:
        return 0.
    else:
        return a*(x**2) + b*x + c + math.log10(k*x)

def weibull(c,b,x):
    a = c * ((-math.log(0.9)) ** (-1/b))
    return 1. - math.exp(-(x / a)**b)

def SpeedMetric(speedUp):
    res = weibull(5, 1.7, speedUp)
    return max(min(res, 1), 0)

def compute_ml_subscore(results, key: str="test_ratio"):
    test_ratio = configuration[key]
    value_by_color = configuration["value_by_color"]
    test_ml_res = sum([value_by_color[color] for color in results["ML"]])
    test_ml_subscore = (test_ml_res * test_ratio["ml"]) / (len(results["ML"])*max(value_by_color.values()))
    return test_ml_subscore

def compute_physics_subscore(results, key: str="test_ratio"):
    test_ratio = configuration[key]
    value_by_color = configuration["value_by_color"]
    test_physics_res = sum([value_by_color[color] for color in results["Physics"]])
    test_physics_subscore = (test_physics_res*test_ratio["physics"]) / (len(results["Physics"])*max(value_by_color.values()))
    return test_physics_subscore

def compute_global_score(metrics, config):
    coefficients = configuration["coefficients"]
    max_speed_ratio_allowed = configuration["max_speed_ratio_allowed"]

    test_metrics = reconstruct_metric_dict(metrics, "test")
    test_ood_metrics = reconstruct_metric_dict(metrics, "test_ood_topo")

    test_results_disc = discretize_results(test_metrics)
    test_ood_results_disc = discretize_results(test_ood_metrics)

    print(test_results_disc)
    print(test_ood_results_disc)

    test_ml_subscore = compute_ml_subscore(test_results_disc, key="test_ratio")
    test_physics_subscore = compute_physics_subscore(test_results_disc, key="test_ratio")
    test_subscore = test_ml_subscore + test_physics_subscore

    test_ood_ml_subscore = compute_ml_subscore(test_ood_results_disc, key="test_ood_ratio")
    test_ood_physics_subscore = compute_physics_subscore(test_ood_results_disc, key="test_ood_ratio")
    test_ood_subscore = test_ood_ml_subscore + test_ood_physics_subscore

    speed_up = compute_speed_up(config, metrics)
    speedup_score = SpeedMetric(speedUp=speed_up)

    globalScore = 100*(coefficients["test"]*test_subscore+coefficients["test_ood"]*test_ood_subscore+coefficients["speed_up"]*speedup_score)
    print('speed_up', speed_up)
    return globalScore


In [ ]:
import os; os.chdir('/content/ml4physim_startingkit_powergrid')
#from utils.compute_score import compute_global_score
score = compute_global_score(metrics, benchmark.config)

# 60.80829635961514 | speed_up 56.25129803310347
# 60.7745165291409  | speed_up 54.022122452863435 | num_layers=5, d_model=512, num_heads=8, dff=512, gelu, 300 epochs
# 57.34453864372906 | speed_up 49.393849658818276 | num_layers=6, d_model=512, num_heads=8, dff=512, gelu
# 54.26904923152882 | speed_up 71.6313714493595   | num_layers=4, d_model=512, num_heads=8, dff=512, gelu

# 48.11191026583244 | speed_up 33.242675980027485 | 8 layer, 512 dim, 8 heads
# 44.65547102334247 | speed_up 26.942914724461087 | 18 layer, 512, 8 heads

score

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import os
import json

def import_metrics(path, dataset):
    path_to_results = os.path.join(path, dataset, "eval_res.json")
    with open(path_to_results) as json_file:
        metrics = json.load(json_file)
    return metrics


EVALUATION_PATH = os.path.join("input_data_local", "eval_results", "lips_idf_2023", "tf_leapnet_DEFAULT")
metrics_dict = dict()
metrics_dict["test"] = import_metrics(EVALUATION_PATH, "test")
metrics_dict["test_ood_topo"] = import_metrics(EVALUATION_PATH, "test_ood_topo")

In [ ]:
from utils.compute_score import compute_ml_subscore, compute_physics_subscore, SpeedMetric, configuration

test_results_disc = {"ML": ['g','g','g','g','g','g'], "Physics": ['g','g','g','g','g','g','g','g']}
test_ood_results_disc = {"ML": ['g','g','g','g','g','g'], "Physics": ['g','g','g','g','g','g','g','g']}
coefficients = configuration["coefficients"]
max_speed_ratio_allowed = configuration["max_speed_ratio_allowed"]

test_ml_subscore = compute_ml_subscore(test_results_disc, key="test_ratio")
test_physics_subscore = compute_physics_subscore(test_results_disc, key="test_ratio")
test_subscore = test_ml_subscore + test_physics_subscore

test_ood_ml_subscore = compute_ml_subscore(test_ood_results_disc, key="test_ood_ratio")
test_ood_physics_subscore = compute_physics_subscore(test_ood_results_disc, key="test_ood_ratio")
test_ood_subscore = test_ood_ml_subscore + test_ood_physics_subscore

speed_up = 1. # LighSim2grid
# speed_up = 3.77 # Security Analysis

speedup_score = SpeedMetric(speedUp=speed_up)

globalScore = 100*(coefficients["test"]*test_subscore+coefficients["test_ood"]*test_ood_subscore+coefficients["speed_up"]*speedup_score)
globalScore

60.27227425101766

# Evaluate

In [ ]:
EVAL_SAVE_PATH = get_path(EVALUATION_PATH, benchmark)
tf_fc_metrics = benchmark.evaluate_simulator(augmented_simulator=tf_fc,
                                              eval_batch_size=128,
                                              dataset="all",
                                              shuffle=False,
                                              save_path=None,
                                              save_predictions=False
                                             )
benchmark_name

## this

In [ ]:
tf_fc_metrics["test"]["ML"]

{'MSE_avg': {'a_or': 2.6493818759918213,
  'a_ex': 5.420886039733887,
  'p_or': 0.006804934702813625,
  'p_ex': 0.0068869031965732574,
  'v_or': 0.008070120587944984,
  'v_ex': 0.0047674900852143764},
 'MAE_avg': {'a_or': 0.7493286728858948,
  'a_ex': 1.069725751876831,
  'p_or': 0.049427714198827744,
  'p_ex': 0.051184628158807755,
  'v_or': 0.04797757416963577,
  'v_ex': 0.03513362631201744},
 'MAPE_avg': {'a_or': 142418783502336.0,
  'a_ex': 225023923060736.0,
  'p_or': 12844478234624.0,
  'p_ex': 12468456783872.0,
  'v_or': 14146263318528.0,
  'v_ex': 7372431425536.0},
 'MAPE_90_avg': {'a_or': 0.0032831775505863343,
  'a_ex': 0.003308613875054752,
  'p_or': 0.0031568914101477246,
  'p_ex': 0.003099215847790477,
  'v_or': 0.0005032335263248121,
  'v_ex': 0.0005270154593007024},
 'TIME_INF': 0.5552118039995548}

In [ ]:
tf_fc_metrics["test"]["Physics"]

{'CURRENT_POS': {'a_or': {'Error': 2052.58740234375,
   'Violation_proportion': 0.00918},
  'a_ex': {'Error': 3096.0048828125, 'Violation_proportion': 0.009395}},
 'VOLTAGE_POS': {'v_or': {'Error': 149.36582946777344,
   'Violation_proportion': 0.00976},
  'v_ex': {'Error': 152.95953369140625, 'Violation_proportion': 0.0107}},
 'LOSS_POS': {'loss_criterion': 1008.51874, 'violation_proportion': 0.146995},
 'DISC_LINES': {'p_or': 1.0,
  'p_ex': 1.0,
  'a_or': 1.0,
  'a_ex': 1.0,
  'v_or': 1.0,
  'v_ex': 1.0,
  'violation_proportion': 1.0},
 'CHECK_LOSS': {'violation_percentage': 0.2},
 'CHECK_GC': {'violation_percentage': 99.68,
  'mae': 0.18690997,
  'wmape': 0.040203024},
 'CHECK_LC': {'violation_percentage': 79.49642857142857,
  'mae': 0.04779689596327288,
  'mape': 0.002251925372985263},
 'CHECK_VOLTAGE_EQ': {'prop_voltages_violation': 0.9918533333333334},
 'CHECK_JOULE_LAW': {'violation_proportion': 0.4368066666666667,
  'mae': 0.013169481248639722,
  'wmape': 0.123023470670271}}

In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"]

{'MSE_avg': {'a_or': 5049.1845703125,
  'a_ex': 7370.595703125,
  'p_or': 13.839800834655762,
  'p_ex': 13.630780220031738,
  'v_or': 0.6086974143981934,
  'v_ex': 0.44855839014053345},
 'MAE_avg': {'a_or': 22.983509063720703,
  'a_ex': 30.61013412475586,
  'p_or': 1.8361631631851196,
  'p_ex': 1.8168829679489136,
  'v_or': 0.47250446677207947,
  'v_ex': 0.3684189021587372},
 'MAPE_avg': {'a_or': 1045393697669120.0,
  'a_ex': 1519075708633088.0,
  'p_or': 94084068802560.0,
  'p_ex': 113208534761472.0,
  'v_or': 86229857075200.0,
  'v_ex': 65442227945472.0},
 'MAPE_90_avg': {'a_or': 0.10463213308522482,
  'a_ex': 0.1056751931571713,
  'p_or': 0.10975987916880706,
  'p_ex': 0.10928244820681124,
  'v_or': 0.00698050520415862,
  'v_ex': 0.007475095965187114},
 'TIME_INF': 0.6810263419993134}

In [ ]:
phys = tf_fc_metrics["test"]["Physics"]

print("1) Current positivity violation:", (phys["CURRENT_POS"]["a_or"]["Violation_proportion"]+phys["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
if phys["VOLTAGE_POS"]:
    print("2) Voltage positivity violation:", (phys["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+phys["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
else:
    print("2) Voltage positivity violation:", 0)
print("3) Loss positivity violation:", phys["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", phys["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", phys["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(phys["CHECK_GC"]["violation_percentage"], phys["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(phys["CHECK_LC"]["violation_percentage"], phys["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", phys["CHECK_VOLTAGE_EQ"]["prop_voltages_violation"])

## b1

In [ ]:
tf_fc_metrics["test"]["ML"] # this, b1

{'MSE_avg': {'a_or': 1.7248914241790771, 'a_ex': 3.2528140544891357},
 'MAE_avg': {'a_or': 0.755136251449585, 'a_ex': 1.0725153684616089},
 'MAPE_avg': {'a_or': 229841953619968.0, 'a_ex': 360194714370048.0},
 'MAPE_90_avg': {'a_or': 0.0028101804701745245, 'a_ex': 0.0027460730909808664},
 'TIME_INF': 0.6930989120000959}

In [ ]:
tf_fc_metrics["test"]["Physics"] # this

{'CURRENT_POS': {'a_or': {'Error': 4648.5322265625,
   'Violation_proportion': 0.01981},
  'a_ex': {'Error': 7252.73681640625, 'Violation_proportion': 0.019355}}}

In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"] # this

{'MSE_avg': {'a_or': 9340.43359375, 'a_ex': 15330.8017578125},
 'MAE_avg': {'a_or': 44.300376892089844, 'a_ex': 59.453521728515625},
 'MAPE_avg': {'a_or': 3.277412448914637e+16, 'a_ex': 4.899427678840422e+16},
 'MAPE_90_avg': {'a_or': 0.20768380483642468, 'a_ex': 0.20786264408335828},
 'TIME_INF': 0.6934249860005366}

In [ ]:
tf_fc_metrics["test"]["ML"] # best, transformer layernorm d_model=dff=512 layers=6 heads=8

{'MSE_avg': {'a_or': 1.784806251525879, 'a_ex': 2.854123830795288},
 'MAE_avg': {'a_or': 0.798844039440155, 'a_ex': 1.0281407833099365},
 'MAPE_avg': {'a_or': 228601379160064.0, 'a_ex': 290862433042432.0},
 'MAPE_90_avg': {'a_or': 0.002680124914613662, 'a_ex': 0.0027341529372285596},
 'TIME_INF': 0.6518575379996037}

In [ ]:
tf_fc_metrics["test"]["Physics"] # best, transformer layernorm d_model=dff=512 layers=6 heads=8

{'CURRENT_POS': {'a_or': {'Error': 8489.2646484375,
   'Violation_proportion': 0.039765},
  'a_ex': {'Error': 10200.7568359375, 'Violation_proportion': 0.03856}}}

In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"] # best, transformer layernorm d_model=dff=512 layers=6 heads=8

{'MSE_avg': {'a_or': 9912.9638671875, 'a_ex': 17243.9375},
 'MAE_avg': {'a_or': 45.83586883544922, 'a_ex': 62.482566833496094},
 'MAPE_avg': {'a_or': 3.4595882112385024e+16, 'a_ex': 5.423528364579226e+16},
 'MAPE_90_avg': {'a_or': 0.21768888555268973, 'a_ex': 0.21705632176991366},
 'TIME_INF': 0.6531008499987365}

## B2

In [ ]:
tf_fc_metrics["test"]["ML"] # this

{'MSE_avg': {'a_or': 3.4163074493408203,
  'a_ex': 7.435972690582275,
  'p_or': 0.00812915526330471,
  'p_ex': 0.007996125146746635,
  'v_or': 0.009718255139887333,
  'v_ex': 0.007805672008544207},
 'MAE_avg': {'a_or': 0.8820344805717468,
  'a_ex': 1.2305068969726562,
  'p_or': 0.05603257939219475,
  'p_ex': 0.055735811591148376,
  'v_or': 0.0538327731192112,
  'v_ex': 0.042450688779354095},
 'MAPE_avg': {'a_or': 156576623099904.0,
  'a_ex': 227041853046784.0,
  'p_or': 12760949719040.0,
  'p_ex': 12954137264128.0,
  'v_or': 21397730492416.0,
  'v_ex': 19181099548672.0},
 'MAPE_90_avg': {'a_or': 0.004189981612305778,
  'a_ex': 0.004246600014431888,
  'p_or': 0.0036684688836049915,
  'p_ex': 0.003685390282287055,
  'v_or': 0.0006132913072607021,
  'v_ex': 0.0006335886259249298},
 'TIME_INF': 0.6836444879991177}

In [ ]:
tf_fc_metrics["test"]["Physics"] # this

{'CURRENT_POS': {'a_or': {'Error': 3108.304443359375,
   'Violation_proportion': 0.01267},
  'a_ex': {'Error': 5611.6328125, 'Violation_proportion': 0.01287}},
 'VOLTAGE_POS': {'v_or': {'Error': 416.2125244140625,
   'Violation_proportion': 0.010835},
  'v_ex': {'Error': 284.34210205078125, 'Violation_proportion': 0.009535}},
 'LOSS_POS': {'loss_criterion': 841.58325, 'violation_proportion': 0.15677},
 'DISC_LINES': {'p_or': 1.0,
  'p_ex': 1.0,
  'a_or': 1.0,
  'a_ex': 1.0,
  'v_or': 1.0,
  'v_ex': 1.0,
  'violation_proportion': 1.0},
 'CHECK_LOSS': {'violation_percentage': 0.24},
 'CHECK_GC': {'violation_percentage': 99.35000000000001,
  'mae': 0.116668105,
  'wmape': 0.02509449},
 'CHECK_LC': {'violation_percentage': 80.79571428571428,
  'mae': 0.05077320722671492,
  'mape': 0.0023921526977303074},
 'CHECK_VOLTAGE_EQ': {'prop_voltages_violation': 0.9936733333333333},
 'CHECK_JOULE_LAW': {'violation_proportion': 0.2858133333333333,
  'mae': 0.00849332081342187,
  'wmape': 0.0803620202

In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"] # this

{'MSE_avg': {'a_or': 4951.73876953125,
  'a_ex': 7253.33837890625,
  'p_or': 12.615303993225098,
  'p_ex': 12.286676406860352,
  'v_or': 0.798795759677887,
  'v_ex': 0.4736838638782501},
 'MAE_avg': {'a_or': 23.74113655090332,
  'a_ex': 31.379329681396484,
  'p_or': 1.825121283531189,
  'p_ex': 1.7927690744400024,
  'v_or': 0.5396459102630615,
  'v_ex': 0.3918887972831726},
 'MAPE_avg': {'a_or': 836157587849216.0,
  'a_ex': 1346239111102464.0,
  'p_or': 91425962196992.0,
  'p_ex': 112682996858880.0,
  'v_or': 81713589911552.0,
  'v_ex': 65628731867136.0},
 'MAPE_90_avg': {'a_or': 0.10341826005645646,
  'a_ex': 0.10392443256988113,
  'p_or': 0.10687728993259207,
  'p_ex': 0.1064772475787747,
  'v_or': 0.007269532575512265,
  'v_ex': 0.007646203201812851},
 'TIME_INF': 0.5293780879992482}

In [ ]:
# this
phys = tf_fc_metrics["test"]["Physics"]

print("1) Current positivity violation:", (phys["CURRENT_POS"]["a_or"]["Violation_proportion"]+phys["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
if phys["VOLTAGE_POS"]:
    print("2) Voltage positivity violation:", (phys["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+phys["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
else:
    print("2) Voltage positivity violation:", 0)
print("3) Loss positivity violation:", phys["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", phys["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", phys["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(phys["CHECK_GC"]["violation_percentage"], phys["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(phys["CHECK_LC"]["violation_percentage"], phys["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", phys["CHECK_VOLTAGE_EQ"]["prop_voltages_violation"])

1) Current positivity violation: 0.01277
2) Voltage positivity violation: 0.010185
3) Loss positivity violation: 0.15677
4) Disconnected lines violation: {'p_or': 1.0, 'p_ex': 1.0, 'a_or': 1.0, 'a_ex': 1.0, 'v_or': 1.0, 'v_ex': 1.0, 'violation_proportion': 1.0}
5) Violation of loss to be between [1,4]% of production: 0.24
6) Violation of global conservation: 99.35000000000001% and its weighted mape: 0.025094490498304367
7) Violation of local conservation: 80.79571428571428% and its weighted mape: 0.0023921526977303074
8) Violation proportion of voltage equality at subs: 0.9936733333333333


In [ ]:
tf_fc_metrics["test"]["ML"] # best

{'MSE_avg': {'a_or': 3.3345768451690674,
  'a_ex': 7.118857383728027,
  'p_or': 0.012164519168436527,
  'p_ex': 0.011876890435814857,
  'v_or': 0.022410036996006966,
  'v_ex': 0.015381082892417908},
 'MAE_avg': {'a_or': 0.938480794429779,
  'a_ex': 1.3300251960754395,
  'p_or': 0.068013496696949,
  'p_ex': 0.06734646111726761,
  'v_or': 0.06679807603359222,
  'v_ex': 0.04887884110212326},
 'MAPE_avg': {'a_or': 160140137332736.0,
  'a_ex': 244198452232192.0,
  'p_or': 14076988096512.0,
  'p_ex': 14900682293248.0,
  'v_or': 43712197427200.0,
  'v_ex': 35290919993344.0},
 'MAPE_90_avg': {'a_or': 0.004435715727755689,
  'a_ex': 0.004490790138938692,
  'p_or': 0.004508825384497544,
  'p_ex': 0.004568100071565232,
  'v_or': 0.0006994158451197991,
  'v_ex': 0.000686637240927417},
 'TIME_INF': 0.5706053100002464}

In [ ]:
tf_fc_metrics["test"]["Physics"] # best

{'CURRENT_POS': {'a_or': {'Error': 2351.00146484375,
   'Violation_proportion': 0.01195},
  'a_ex': {'Error': 6405.8076171875, 'Violation_proportion': 0.012055}},
 'VOLTAGE_POS': {'v_or': {'Error': 885.3673095703125,
   'Violation_proportion': 0.01359},
  'v_ex': {'Error': 627.9515380859375, 'Violation_proportion': 0.013135}},
 'LOSS_POS': {'loss_criterion': 1540.0264, 'violation_proportion': 0.202105},
 'DISC_LINES': {'p_or': 1.0,
  'p_ex': 1.0,
  'a_or': 1.0,
  'a_ex': 1.0,
  'v_or': 1.0,
  'v_ex': 1.0,
  'violation_proportion': 1.0},
 'CHECK_LOSS': {'violation_percentage': 0.12},
 'CHECK_GC': {'violation_percentage': 99.51,
  'mae': 0.15969355,
  'wmape': 0.03434896},
 'CHECK_LC': {'violation_percentage': 85.33714285714285,
  'mae': 0.06889437722883054,
  'mape': 0.003245921999973283},
 'CHECK_VOLTAGE_EQ': {'prop_voltages_violation': 0.9899733333333334},
 'CHECK_JOULE_LAW': {'violation_proportion': 0.36358666666666667,
  'mae': 0.011123282501521616,
  'wmape': 0.1066430275099315}}

In [ ]:
phys = {'CURRENT_POS': {'a_or': {'Error': 2351.00146484375,
   'Violation_proportion': 0.01195},
  'a_ex': {'Error': 6405.8076171875, 'Violation_proportion': 0.012055}},
 'VOLTAGE_POS': {'v_or': {'Error': 885.3673095703125,
   'Violation_proportion': 0.01359},
  'v_ex': {'Error': 627.9515380859375, 'Violation_proportion': 0.013135}},
 'LOSS_POS': {'loss_criterion': 1540.0264, 'violation_proportion': 0.202105},
 'DISC_LINES': {'p_or': 1.0,
  'p_ex': 1.0,
  'a_or': 1.0,
  'a_ex': 1.0,
  'v_or': 1.0,
  'v_ex': 1.0,
  'violation_proportion': 1.0},
 'CHECK_LOSS': {'violation_percentage': 0.12},
 'CHECK_GC': {'violation_percentage': 99.51,
  'mae': 0.15969355,
  'wmape': 0.03434896},
 'CHECK_LC': {'violation_percentage': 85.33714285714285,
  'mae': 0.06889437722883054,
  'mape': 0.003245921999973283},
 'CHECK_VOLTAGE_EQ': {'prop_voltages_violation': 0.9899733333333334},
 'CHECK_JOULE_LAW': {'violation_proportion': 0.36358666666666667,
  'mae': 0.011123282501521616,
  'wmape': 0.1066430275099315}}

In [ ]:
print("1) Current positivity violation:", (phys["CURRENT_POS"]["a_or"]["Violation_proportion"]+phys["CURRENT_POS"]["a_ex"]["Violation_proportion"])/2)#["a_or"]["Violation_proportion"]
if phys["VOLTAGE_POS"]:
    print("2) Voltage positivity violation:", (phys["VOLTAGE_POS"]["v_or"]["Violation_proportion"]+phys["VOLTAGE_POS"]["v_ex"]["Violation_proportion"])/2)
else:
    print("2) Voltage positivity violation:", 0)
print("3) Loss positivity violation:", phys["LOSS_POS"]["violation_proportion"])
print("4) Disconnected lines violation:", phys["DISC_LINES"])
print("5) Violation of loss to be between [1,4]% of production:", phys["CHECK_LOSS"]["violation_percentage"])
print("6) Violation of global conservation: {}% and its weighted mape: {}".format(phys["CHECK_GC"]["violation_percentage"], phys["CHECK_GC"]["wmape"]))
print("7) Violation of local conservation: {}% and its weighted mape: {}".format(phys["CHECK_LC"]["violation_percentage"], phys["CHECK_LC"]["mape"]))
print("8) Violation proportion of voltage equality at subs:", phys["CHECK_VOLTAGE_EQ"]["prop_voltages_violation"])

1) Current positivity violation: 0.0120025
2) Voltage positivity violation: 0.0133625
3) Loss positivity violation: 0.202105
4) Disconnected lines violation: {'p_or': 1.0, 'p_ex': 1.0, 'a_or': 1.0, 'a_ex': 1.0, 'v_or': 1.0, 'v_ex': 1.0, 'violation_proportion': 1.0}
5) Violation of loss to be between [1,4]% of production: 0.12
6) Violation of global conservation: 99.51% and its weighted mape: 0.03434896
7) Violation of local conservation: 85.33714285714285% and its weighted mape: 0.003245921999973283
8) Violation proportion of voltage equality at subs: 0.9899733333333334


In [ ]:
tf_fc_metrics["test_ood_topo"]["ML"] # best

{'MSE_avg': {'a_or': 5894.8837890625,
  'a_ex': 8586.091796875,
  'p_or': 16.85958480834961,
  'p_ex': 16.473779678344727,
  'v_or': 0.8979676365852356,
  'v_ex': 0.5282770991325378},
 'MAE_avg': {'a_or': 28.1899356842041,
  'a_ex': 36.7491455078125,
  'p_or': 2.171426296234131,
  'p_ex': 2.154911756515503,
  'v_or': 0.5106374025344849,
  'v_ex': 0.3797556161880493},
 'MAPE_avg': {'a_or': 1004337971068928.0,
  'a_ex': 1590081047494656.0,
  'p_or': 118584684576768.0,
  'p_ex': 145763690610688.0,
  'v_or': 149274792820736.0,
  'v_ex': 124617226190848.0},
 'MAPE_90_avg': {'a_or': 0.12828775049733776,
  'a_ex': 0.12835795985201354,
  'p_or': 0.13189085260852623,
  'p_ex': 0.13114805391786594,
  'v_or': 0.007774807833734673,
  'v_ex': 0.00811277348159304},
 'TIME_INF': 0.6962272950004262}

In [ ]:
from google.colab import runtime
runtime.unassign()